In [5]:
import pandas as  pd 
import numpy as np 
import matplotlib.pyplot as plt 
from  datetime import datetime

In [6]:
# load data 
train=pd.read_csv("train.csv")
sample_sub=pd.read_csv("Sample_sub.csv")

In [7]:
sample_sub.head()

,ID,Capture_Number
0,CaptureSite_0_201901,7
1,CaptureSite_0_201902,1
2,CaptureSite_0_201903,5
3,CaptureSite_0_201904,2
4,CaptureSite_0_201905,3


In [8]:
# split Id to Year woy and CaptureSite
sample_sub["year_woy"]=(sample_sub.ID.apply(lambda x: x.split("_")[-1])).astype(int)
sample_sub["CaptureSite"]=sample_sub.ID.apply(lambda x: ("_").join(x.split("_")[0:-1]))

In [9]:
# Create Time features from Date_TimeCaught 
train["Date_TimeCaught"]=pd.to_datetime(train["Date_TimeCaught"])
print(train.Date_TimeCaught.min(),train.Date_TimeCaught.max())
train["year"]=train.Date_TimeCaught.dt.year
train["week_of_year"]=train.Date_TimeCaught.dt.weekofyear
train["year_woy"]=train.year*100+train.week_of_year

1998-04-17 00:00:00 2018-12-31 00:00:00


In [10]:
keys=pd.concat([train[["year_woy","CaptureSite"]],sample_sub[["year_woy","CaptureSite"]]])
# keys.sort_values(["CaptureSite","year_woy"],inplace=True)
CaptureSite_min_year_woy=keys.groupby("CaptureSite").year_woy.min().rename("year_woy").reset_index()
CaptureSite_min_year_woy.head()

,CaptureSite,year_woy
0,CaptureSite_0,200051
1,CaptureSite_1,200123
2,CaptureSite_10,200050
3,CaptureSite_11,199828
4,CaptureSite_12,200107


In [11]:
range_year_woy=pd.DataFrame()
range_year_woy["Date_TimeCaught"]=pd.date_range(start=train.Date_TimeCaught.min(),end=datetime(2019,10,31))
range_year_woy["year"]=range_year_woy.Date_TimeCaught.dt.year
range_year_woy["week_of_year"]=range_year_woy.Date_TimeCaught.dt.weekofyear
range_year_woy["year_woy"]=range_year_woy.year*100+range_year_woy.week_of_year
range_year_woy.drop_duplicates(["year_woy"],inplace=True)

In [12]:
final_data=[]
for site , year_woy in zip(CaptureSite_min_year_woy.CaptureSite.values,CaptureSite_min_year_woy.year_woy.values) :

    one_site_df=range_year_woy[range_year_woy.year_woy>=year_woy]
    one_site_df["CaptureSite"]=site
    final_data.append(one_site_df)
final_data=pd.concat(final_data)
len(final_data)

D:\condaa\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


28048

In [13]:
Target=train.groupby(["year_woy","CaptureSite"]).CaptureSite.count().rename("Capture_Number").reset_index()
final_data=final_data.merge(Target,on=["year_woy","CaptureSite"],how="left")
final_data.Capture_Number.fillna(0,inplace=True)

In [14]:
final_data.Capture_Number.fillna(0).value_counts(True)

0.0     0.716308
1.0     0.147461
2.0     0.059469
3.0     0.030626
4.0     0.015759
5.0     0.009199
6.0     0.005990
7.0     0.005455
8.0     0.002603
9.0     0.002104
10.0    0.001604
11.0    0.000998
12.0    0.000606
14.0    0.000535
13.0    0.000499
15.0    0.000214
16.0    0.000178
17.0    0.000107
28.0    0.000036
22.0    0.000036
19.0    0.000036
35.0    0.000036
20.0    0.000036
21.0    0.000036
18.0    0.000036
23.0    0.000036
Name: Capture_Number, dtype: float64

In [15]:
train=final_data[final_data.year<2019]
test=final_data[final_data.year==2019]

In [16]:
#Remove data with year < 2010

In [17]:
train=train[train['year']>2010]

In [18]:
train

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number
526,2011-01-01,2011,52,201152,CaptureSite_0,0.0
527,2011-01-03,2011,1,201101,CaptureSite_0,0.0
528,2011-01-10,2011,2,201102,CaptureSite_0,0.0
529,2011-01-17,2011,3,201103,CaptureSite_0,0.0
530,2011-01-24,2011,4,201104,CaptureSite_0,0.0
...,...,...,...,...,...,...
27999,2018-11-26,2018,48,201848,CaptureSite_9,6.0
28000,2018-12-03,2018,49,201849,CaptureSite_9,6.0
28001,2018-12-10,2018,50,201850,CaptureSite_9,12.0
28002,2018-12-17,2018,51,201851,CaptureSite_9,5.0


In [19]:
train["Date_TimeCaught"]

526     2011-01-01
527     2011-01-03
528     2011-01-10
529     2011-01-17
530     2011-01-24
           ...    
27999   2018-11-26
28000   2018-12-03
28001   2018-12-10
28002   2018-12-17
28003   2018-12-24
Name: Date_TimeCaught, Length: 12122, dtype: datetime64[ns]

In [20]:
aux=pd.get_dummies(train['CaptureSite'])
train=pd.concat([train,aux],axis=1)
auxtest=pd.get_dummies(test['CaptureSite'])
test=pd.concat([test,auxtest],axis=1)

In [21]:
#for each CaptureSite add the mean of the Capture_Number

In [22]:
trainMean=train.groupby('CaptureSite').mean()[['Capture_Number']].reset_index()

trainMean.columns=['CaptureSite','moyenne']
train=pd.merge(train,trainMean,on=['CaptureSite'],how='left')
test=pd.merge(test,trainMean,on=['CaptureSite'],how='left')

In [23]:
train

,Date_TimeCaught,year,week_of_year,year_woy,CaptureSite,Capture_Number,CaptureSite_0,CaptureSite_1,CaptureSite_10,CaptureSite_11,...,CaptureSite_27,CaptureSite_28,CaptureSite_3,CaptureSite_4,CaptureSite_5,CaptureSite_6,CaptureSite_7,CaptureSite_8,CaptureSite_9,moyenne
0,2011-01-01,2011,52,201152,CaptureSite_0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.148325
1,2011-01-03,2011,1,201101,CaptureSite_0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.148325
2,2011-01-10,2011,2,201102,CaptureSite_0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.148325
3,2011-01-17,2011,3,201103,CaptureSite_0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.148325
4,2011-01-24,2011,4,201104,CaptureSite_0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.148325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12117,2018-11-26,2018,48,201848,CaptureSite_9,6.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.538278
12118,2018-12-03,2018,49,201849,CaptureSite_9,6.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.538278
12119,2018-12-10,2018,50,201850,CaptureSite_9,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.538278
12120,2018-12-17,2018,51,201851,CaptureSite_9,5.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.538278


In [24]:
old=pd.read_csv("train.csv")

In [25]:
old.columns

Index(['Rescue_ID', 'Date_TimeCaught', 'Researcher', 'CaptureSite',
       'ForagingGround', 'CaptureMethod', 'Fisher', 'LandingSite', 'Species',
       'Tag_1', 'Tag_2', 'Lost_Tags', 'T_Number', 'CCL_cm', 'CCW_cm',
       'Weight_Kg', 'Sex', 'TurtleCharacteristics', 'Status', 'ReleaseSite',
       'Date_TimeRelease'],
      dtype='object')

In [26]:
#add the number of unique researcher and species for each CaptureSite

In [27]:
new=old[['Researcher','Species',
      'CaptureSite' ]].groupby('CaptureSite').nunique()

In [28]:
new.drop(columns='CaptureSite',axis=1,inplace=True)
new=new.reset_index()

In [29]:
#period of the year

In [30]:
train['periode']=train['week_of_year']/9
test['periode']=test['week_of_year']/9

In [31]:
test=pd.merge(test,new,on=['CaptureSite'],how='left')
train=pd.merge(train,new,on=['CaptureSite'],how='left')

In [32]:
#for each CaptureSite calculate the number of participation for all fishers 

In [33]:
fisher=pd.get_dummies(old['Fisher'])

In [34]:
fisher=pd.concat([old[['CaptureSite']],fisher],axis=1)

In [35]:
fisher=fisher.groupby('CaptureSite').count()

In [36]:
test=pd.merge(test,fisher,on=['CaptureSite'],how='left')
train=pd.merge(train,fisher,on=['CaptureSite'],how='left')

In [37]:
test.drop(columns=['Date_TimeCaught','Capture_Number','CaptureSite'],inplace=True)
train.drop(columns=['Date_TimeCaught','CaptureSite'],inplace=True)

In [38]:

from catboost import Pool, CatBoostRegressor
cbm0 = CatBoostRegressor()
X,y=train.drop(columns='Capture_Number',axis=1),train['Capture_Number']

In [39]:
sub=pd.read_csv('Sample_sub.csv')

In [40]:
cbm0.fit(X,y)
res=cbm0.predict(test)


0:	learn: 1.8672403	total: 312ms	remaining: 5m 11s
1:	learn: 1.8479780	total: 459ms	remaining: 3m 48s
2:	learn: 1.8297886	total: 584ms	remaining: 3m 14s
3:	learn: 1.8125172	total: 693ms	remaining: 2m 52s
4:	learn: 1.7960103	total: 797ms	remaining: 2m 38s
5:	learn: 1.7804202	total: 916ms	remaining: 2m 31s
6:	learn: 1.7654272	total: 1.05s	remaining: 2m 29s
7:	learn: 1.7512990	total: 1.14s	remaining: 2m 21s
8:	learn: 1.7379640	total: 1.25s	remaining: 2m 18s
9:	learn: 1.7252201	total: 1.37s	remaining: 2m 15s
10:	learn: 1.7130716	total: 1.48s	remaining: 2m 12s
11:	learn: 1.7016030	total: 1.6s	remaining: 2m 11s
12:	learn: 1.6907553	total: 1.7s	remaining: 2m 9s
13:	learn: 1.6804915	total: 1.8s	remaining: 2m 6s
14:	learn: 1.6708258	total: 1.89s	remaining: 2m 4s
15:	learn: 1.6615370	total: 1.99s	remaining: 2m 2s
16:	learn: 1.6528391	total: 2.1s	remaining: 2m 1s
17:	learn: 1.6438825	total: 2.2s	remaining: 1m 59s
18:	learn: 1.6360174	total: 2.32s	remaining: 1m 59s
19:	learn: 1.6286401	total: 2.43

158:	learn: 1.4436917	total: 17.9s	remaining: 1m 34s
159:	learn: 1.4432374	total: 18.1s	remaining: 1m 34s
160:	learn: 1.4428410	total: 18.2s	remaining: 1m 34s
161:	learn: 1.4424372	total: 18.3s	remaining: 1m 34s
162:	learn: 1.4420530	total: 18.4s	remaining: 1m 34s
163:	learn: 1.4415043	total: 18.5s	remaining: 1m 34s
164:	learn: 1.4413599	total: 18.6s	remaining: 1m 34s
165:	learn: 1.4411816	total: 18.7s	remaining: 1m 34s
166:	learn: 1.4408292	total: 18.8s	remaining: 1m 33s
167:	learn: 1.4403622	total: 18.9s	remaining: 1m 33s
168:	learn: 1.4398167	total: 19s	remaining: 1m 33s
169:	learn: 1.4395308	total: 19.1s	remaining: 1m 33s
170:	learn: 1.4392728	total: 19.2s	remaining: 1m 33s
171:	learn: 1.4389486	total: 19.4s	remaining: 1m 33s
172:	learn: 1.4385706	total: 19.5s	remaining: 1m 33s
173:	learn: 1.4380600	total: 19.6s	remaining: 1m 32s
174:	learn: 1.4378364	total: 19.7s	remaining: 1m 32s
175:	learn: 1.4371660	total: 19.8s	remaining: 1m 32s
176:	learn: 1.4370954	total: 19.9s	remaining: 1m

315:	learn: 1.3982822	total: 34.9s	remaining: 1m 15s
316:	learn: 1.3978406	total: 35s	remaining: 1m 15s
317:	learn: 1.3973811	total: 35.2s	remaining: 1m 15s
318:	learn: 1.3970384	total: 35.3s	remaining: 1m 15s
319:	learn: 1.3969038	total: 35.4s	remaining: 1m 15s
320:	learn: 1.3962633	total: 35.5s	remaining: 1m 14s
321:	learn: 1.3960090	total: 35.6s	remaining: 1m 14s
322:	learn: 1.3955536	total: 35.6s	remaining: 1m 14s
323:	learn: 1.3950242	total: 35.7s	remaining: 1m 14s
324:	learn: 1.3947103	total: 35.8s	remaining: 1m 14s
325:	learn: 1.3942491	total: 35.9s	remaining: 1m 14s
326:	learn: 1.3940987	total: 36s	remaining: 1m 14s
327:	learn: 1.3937318	total: 36.1s	remaining: 1m 14s
328:	learn: 1.3935155	total: 36.2s	remaining: 1m 13s
329:	learn: 1.3929872	total: 36.3s	remaining: 1m 13s
330:	learn: 1.3923866	total: 36.4s	remaining: 1m 13s
331:	learn: 1.3916473	total: 36.5s	remaining: 1m 13s
332:	learn: 1.3914980	total: 36.6s	remaining: 1m 13s
333:	learn: 1.3911272	total: 36.7s	remaining: 1m 1

475:	learn: 1.3410198	total: 52s	remaining: 57.2s
476:	learn: 1.3408515	total: 52.1s	remaining: 57.1s
477:	learn: 1.3406025	total: 52.2s	remaining: 57s
478:	learn: 1.3403096	total: 52.3s	remaining: 56.9s
479:	learn: 1.3398989	total: 52.4s	remaining: 56.8s
480:	learn: 1.3395909	total: 52.5s	remaining: 56.7s
481:	learn: 1.3394204	total: 52.6s	remaining: 56.6s
482:	learn: 1.3391325	total: 52.7s	remaining: 56.5s
483:	learn: 1.3389611	total: 52.8s	remaining: 56.3s
484:	learn: 1.3384224	total: 53s	remaining: 56.2s
485:	learn: 1.3381924	total: 53.1s	remaining: 56.1s
486:	learn: 1.3380715	total: 53.2s	remaining: 56s
487:	learn: 1.3379007	total: 53.3s	remaining: 55.9s
488:	learn: 1.3373903	total: 53.3s	remaining: 55.7s
489:	learn: 1.3371926	total: 53.4s	remaining: 55.6s
490:	learn: 1.3366335	total: 53.5s	remaining: 55.5s
491:	learn: 1.3363221	total: 53.6s	remaining: 55.4s
492:	learn: 1.3361470	total: 53.8s	remaining: 55.3s
493:	learn: 1.3357431	total: 53.9s	remaining: 55.2s
494:	learn: 1.335472

637:	learn: 1.3047473	total: 1m 8s	remaining: 38.9s
638:	learn: 1.3045515	total: 1m 8s	remaining: 38.8s
639:	learn: 1.3043167	total: 1m 8s	remaining: 38.7s
640:	learn: 1.3042064	total: 1m 8s	remaining: 38.5s
641:	learn: 1.3041154	total: 1m 8s	remaining: 38.4s
642:	learn: 1.3039112	total: 1m 9s	remaining: 38.3s
643:	learn: 1.3036032	total: 1m 9s	remaining: 38.2s
644:	learn: 1.3035240	total: 1m 9s	remaining: 38.1s
645:	learn: 1.3033567	total: 1m 9s	remaining: 38s
646:	learn: 1.3031053	total: 1m 9s	remaining: 37.9s
647:	learn: 1.3029605	total: 1m 9s	remaining: 37.8s
648:	learn: 1.3028427	total: 1m 9s	remaining: 37.7s
649:	learn: 1.3027567	total: 1m 9s	remaining: 37.6s
650:	learn: 1.3025873	total: 1m 9s	remaining: 37.5s
651:	learn: 1.3024813	total: 1m 9s	remaining: 37.4s
652:	learn: 1.3023377	total: 1m 10s	remaining: 37.2s
653:	learn: 1.3020156	total: 1m 10s	remaining: 37.1s
654:	learn: 1.3019001	total: 1m 10s	remaining: 37s
655:	learn: 1.3018042	total: 1m 10s	remaining: 36.9s
656:	learn: 

794:	learn: 1.2791401	total: 1m 25s	remaining: 21.9s
795:	learn: 1.2789755	total: 1m 25s	remaining: 21.8s
796:	learn: 1.2789165	total: 1m 25s	remaining: 21.7s
797:	learn: 1.2788246	total: 1m 25s	remaining: 21.6s
798:	learn: 1.2787242	total: 1m 25s	remaining: 21.5s
799:	learn: 1.2785784	total: 1m 25s	remaining: 21.4s
800:	learn: 1.2783796	total: 1m 25s	remaining: 21.3s
801:	learn: 1.2782220	total: 1m 25s	remaining: 21.2s
802:	learn: 1.2780833	total: 1m 25s	remaining: 21.1s
803:	learn: 1.2778548	total: 1m 26s	remaining: 21s
804:	learn: 1.2777698	total: 1m 26s	remaining: 20.9s
805:	learn: 1.2776318	total: 1m 26s	remaining: 20.8s
806:	learn: 1.2774705	total: 1m 26s	remaining: 20.7s
807:	learn: 1.2772914	total: 1m 26s	remaining: 20.6s
808:	learn: 1.2771153	total: 1m 26s	remaining: 20.4s
809:	learn: 1.2770534	total: 1m 26s	remaining: 20.3s
810:	learn: 1.2769186	total: 1m 26s	remaining: 20.2s
811:	learn: 1.2768046	total: 1m 26s	remaining: 20.1s
812:	learn: 1.2765852	total: 1m 27s	remaining: 2

951:	learn: 1.2577842	total: 1m 41s	remaining: 5.13s
952:	learn: 1.2577157	total: 1m 41s	remaining: 5.03s
953:	learn: 1.2576709	total: 1m 42s	remaining: 4.92s
954:	learn: 1.2574567	total: 1m 42s	remaining: 4.81s
955:	learn: 1.2574054	total: 1m 42s	remaining: 4.71s
956:	learn: 1.2572798	total: 1m 42s	remaining: 4.6s
957:	learn: 1.2572325	total: 1m 42s	remaining: 4.49s
958:	learn: 1.2570908	total: 1m 42s	remaining: 4.38s
959:	learn: 1.2570448	total: 1m 42s	remaining: 4.28s
960:	learn: 1.2568992	total: 1m 42s	remaining: 4.17s
961:	learn: 1.2568148	total: 1m 42s	remaining: 4.06s
962:	learn: 1.2566723	total: 1m 43s	remaining: 3.96s
963:	learn: 1.2566287	total: 1m 43s	remaining: 3.85s
964:	learn: 1.2565140	total: 1m 43s	remaining: 3.74s
965:	learn: 1.2564010	total: 1m 43s	remaining: 3.64s
966:	learn: 1.2562427	total: 1m 43s	remaining: 3.53s
967:	learn: 1.2561652	total: 1m 43s	remaining: 3.42s
968:	learn: 1.2559514	total: 1m 43s	remaining: 3.32s
969:	learn: 1.2557873	total: 1m 43s	remaining: 

In [41]:

from lightgbm import LGBMRegressor
model = LGBMRegressor(max_depth=4,learning_rate=0.17,n_estimators=1000,min_child_weight =80,colsample_bytree=0.5,random_state=50,colsample_bylevel=0.6)
model.fit(X,y)
pred = model.predict(test)

In [42]:
# final_resut=catboost*0.62+lightgbm*0.38
pred=0.38*pred+0.62*res
sub['Capture_Number']=pred

In [43]:
sub.to_csv('brag.csv',index=False)

In [44]:
sub

,ID,Capture_Number
0,CaptureSite_0_201901,0.388788
1,CaptureSite_0_201902,0.227685
2,CaptureSite_0_201903,0.279881
3,CaptureSite_0_201904,0.297893
4,CaptureSite_0_201905,0.384514
...,...,...
1271,CaptureSite_9_201940,3.898697
1272,CaptureSite_9_201941,4.359014
1273,CaptureSite_9_201942,4.298171
1274,CaptureSite_9_201943,4.140034
